In [0]:
!pip install cupy-cuda101

In [0]:
import itertools
import numpy as np
import cupy as cp 

In [29]:
# Последовательности для выравнивания
A = "GGTTGTTA"
B = "TGGACGG"

# Всё заглавными буквами
max(scores)

# Очки и штрафы
m_score = 2  # Очки при совпадении нуклеотидов
mm_penalty = 0  # Штраф за несовпадение
gap_penalty = -1  # Штраф за гэп

# Инициализация матриц
H = cp.zeros((len(A) + 1, len(B) + 1), dtype=np.float)  # Матрица с очками
cell_types = cp.zeros((len(A) + 1, len(B) + 1), dtype=np.int)  # вспомогат.

# Заполенение матрицы с очками
for i, j in itertools.product(range(1, len(A) + 1), range(1, len(B) + 1)):
    if A[i-1] == B[j-1]:
        match = H[i-1, j-1] + m_score  # мэтч
    else:
        match = H[i-1, j-1] + mm_penalty  # мисмэтч

    insertion = H[i, j-1] + gap_penalty  # инсерция
    deletion = H[i-1, j] + gap_penalty  # делеция
    H[i, j] = max(0, match, insertion, deletion)  # заполнение ячейки
    # В отдельную матрицу "тип" ячейки (мэтч, инсерция, делеция или ничто)
    cell_types[i, j] = np.argmax([0, match, insertion, deletion])

# Ячейки с максимальным количеством очков в них
Hnp = cp.asnumpy(H)
starting_cells = cp.array(np.argwhere(Hnp == np.amax(Hnp)))

alignments = []  # список для выравниваний
scores = []  # список для суммарных очков выравниваний

# Теперь идём по максимальным значениям ячеек и выписываем выравнивания
for cell in starting_cells:
    i, j = cell  # положение ячейки
    i = int(i)
    j = int(j)

    cell_score = H[i, j]  # очки в ячейке
    stack = []  # список для пар нуклеотидов
    score = 0  # переменная для суммарного score выравнивания

    while cell_score > 0:
        score += cell_score

        if cell_types[i, j] == 1:
            base1 = A[i-1]
            base2 = B[j-1]
            i -= 1
            j -= 1

        elif cell_types[i, j] == 2:
            base1 = '-'
            base2 = B[j-1]
            i -= 1

        elif cell_types[i, j] == 3:
            base1 = A[i-1]
            base2 = '-'
            j -= 1

        stack.append([base1, base2])
        cell_score = H[i, j]

    scores.append(score)
    a1 = ''
    a2 = ''

    for pair in stack:
        a1 = a1 + pair[0]
        a2 = a2 + pair[1]

    alignments.append([a1[::-1], a2[::-1]])

# выписывание наилучшего выравнивания
print('Наилучшие выравнивания:')
for i in range(len(alignments)):
    if scores[i] == max(scores):
        print('Score = ' + str(scores[i]))
        print(alignments[i][0])
        print(alignments[i][1])

Наилучшие выравнивания:
Score = 26.0
GGTTGT
GGACGG
